In [1]:
from opf_ensemble import OpfSemble
import numpy as np
import sys

from sklearn.metrics import accuracy_score, recall_score, f1_score
from opfython.models.unsupervised import UnsupervisedOPF

opf_ens = OpfSemble(n_models=10, n_folds=10)

Creating model 1
Creating model  2
Creating model  3
Creating model  4
Creating model  5
Creating model  6
Creating model  7
Creating model  8
Creating model  9
Creating model  10


In [9]:
train = np.loadtxt('data/vertebral_column/2/train.txt',delimiter=',', dtype=np.float32)
valid = np.loadtxt('data/vertebral_column/2/valid.txt',delimiter=',', dtype=np.float32)
test = np.loadtxt('data/vertebral_column/2/test.txt',delimiter=',', dtype=np.float32)

X = train[:,:-1]
y = train[:,-1].astype(np.int) 

X_valid = valid[:,:-1]
y_valid = valid[:,-1].astype(np.int) 

X_test = test[:,:-1]
y_test = test[:,-1].astype(np.int) 

new_x = opf_ens.fit(X, y)

In [12]:
np.set_printoptions(threshold=sys.maxsize)


k_max = [5,10,20,30,40,50]
best_k = 0
value_best_k = -1.0

for k in k_max:
    opf_ens.fit_meta_model(new_x,k)
    value = f1_score(opf_ens.predict(X_valid),y_valid)
    if value>value_best_k:
        best_k = k
        value_best_k = value        
print(best_k)
opf_ens.fit_meta_model(new_x,best_k)
pred_ensamble = opf_ens.predict(X_test)

5


In [13]:
print('{} = {}\n'.format('Ensamble',f1_score(pred_ensamble,y_test)))
for key in opf_ens.ensemble:
    model = opf_ens.ensemble[key]
    pred = model.predict(X_test)
    #res = (yhat == test_y).astype(int)
    print('{} = {}'.format(key,f1_score(pred,y_test)))  

Ensamble = 0.9411764705882353

OPF_1 = 0.8571428571428571
Extra Trees_1 = 0.875
Random Forest_1 = 0.8888888888888888
KNN_1 = 0.9253731343283582
KNN_2 = 0.9411764705882353
KNN_3 = 0.9253731343283582
KNN_4 = 0.9411764705882353
Extra Trees_2 = 0.8923076923076922
KNN_5 = 0.8787878787878787
Random Forest_2 = 0.8888888888888888
